# 3. FUNCTIONAL DEPENDENCIES

A Functional Dependency (FD), written as X → Y, states that in a relation R the values of the attributes in X uniquely determine the values of the attributes in Y. This means that whenever two tuples have the same values on X, they must also have the same values on Y.

The goal of functional dependency discovery is to find minimal and completely non-trivial dependencies. A FD is completely non-trivial when the attributes on the right hand side do not appear on the left hand side. A FD is minimal when the right hand side does not depend on any proper subset of the left hand side.


In this notebook the definition of functional dependency is first applied manually to the Milan public establishments dataset, using simple pandas operations.

The focus is on a few natural address-related dependencies, and then the same attributes are used as input for FD discovery algorithms such as TANE, CTANE and FD_Mine.


In [28]:
import pandas as pd

MILANO = pd.read_csv("Comune-di-Milano-Pubblici-esercizi(in)-2.csv", sep=";")
MILANO.head()


,þÿTipo esercizio storico pe,Insegna,Ubicazione,Tipo via,Descrizione via,Civico,Codice via,ZD,Forma commercio,Forma commercio prev,Forma vendita,Settore storico pe,Superficie somministrazione
0,NaN,NaN,ALZ NAVIGLIO GRANDE N. 12 ; isolato:057; (z.d. 6),ALZ,NAVIGLIO GRANDE,12,5144,6,NaN,NaN,NaN,"Ristorante, trattoria, osteria;Genere Merceol....",83.0
1,NaN,NaN,ALZ NAVIGLIO GRANDE N. 44 (z.d. 6),ALZ,NAVIGLIO GRANDE,44,5144,6,NaN,NaN,NaN,Bar gastronomici e simili,26.0
2,NaN,NaN,ALZ NAVIGLIO GRANDE N. 48 (z.d. 6),ALZ,NAVIGLIO GRANDE,48,5144,6,NaN,NaN,NaN,Bar gastronomici e simili,58.0
3,NaN,NaN,ALZ NAVIGLIO GRANDE N. 8 (z.d. 6),ALZ,NAVIGLIO GRANDE,8,5144,6,NaN,NaN,NaN,"BAR CAFFÿý E SIMILI;Ristorante, trattoria, ost...",101.0
4,NaN,NaN,ALZ NAVIGLIO PAVESE N. 24 (z.d. 6),ALZ,NAVIGLIO PAVESE,24,5161,6,NaN,NaN,NaN,Bar gastronomici e simili,51.0


To test candidate functional dependencies on the Milan data, a small helper function is defined.

Given a left hand side X (a list of columns) and a right hand side Y (a single column), the function groups the data by X and counts how many distinct values of Y appear inside each group.

If every group contains exactly one distinct value of Y, then X → Y holds exactly in the data. If some groups contain more than one value of Y, the function reports how many groups violate the dependency and estimates how many rows are involved in the violations.


In [29]:
def check_fd(df, lhs_cols, rhs_col):
    grouped = df.groupby(lhs_cols)[rhs_col].nunique()
    violating_groups = grouped[grouped > 1]

    total_groups = len(grouped)
    num_violating_groups = len(violating_groups)

    if num_violating_groups == 0:
        print(f"FD {lhs_cols} -> {rhs_col} holds exactly.")
        return None
    else:
        print(f"FD {lhs_cols} -> {rhs_col} is violated in {num_violating_groups} out of {total_groups} groups.")
        violating_rows = (
            df.set_index(lhs_cols)
              .index.isin(violating_groups.index)
              .sum()
        )
        total_rows = len(df)
        ratio = violating_rows / total_rows
        print(f"Approximately {violating_rows} rows ({ratio:.2%}) are involved in violations.")
        return violating_groups.sort_values(ascending=False)


A first natural candidate functional dependency on the Milan dataset is that each combination of street type and street name should correspond to a unique street code.

Formally this can be written as:

{`Tipo via`, `Descrizione via`} → `Codice via`

The helper function is used to check whether this dependency holds exactly in the Milan data.


In [30]:
fd1_violations = check_fd(
    MILANO,
    lhs_cols=["Tipo via", "Descrizione via"],
    rhs_col="Codice via",
)

fd1_violations


FD ['Tipo via', 'Descrizione via'] -> Codice via holds exactly.


If the pair (`Tipo via`, `Descrizione via`) determines a unique `Codice via`, it is also reasonable to expect the reverse direction: a given street code should always correspond to a single street type and a single street name.

This leads to two separate functional dependencies:

`Codice via` → `Tipo via`  
`Codice via` → `Descrizione via`

The same helper function is used to verify these dependencies on the Milan dataset.


In [31]:
fd2_violations = check_fd(
    MILANO,
    lhs_cols=["Codice via"],
    rhs_col="Tipo via",
)

fd3_violations = check_fd(
    MILANO,
    lhs_cols=["Codice via"],
    rhs_col="Descrizione via",
)

fd2_violations, fd3_violations


FD ['Codice via'] -> Tipo via holds exactly.
FD ['Codice via'] -> Descrizione via holds exactly.


(None, None)

A more ambitious candidate dependency involves the full address and the zone information.

The intuition is that the combination of street type, street name, civic number, street code and zone should determine a single textual location. This can be written as:

{`Tipo via`, `Descrizione via`, `Civico`, `Codice via`, `ZD`} → `Ubicazione`

The helper function is used again, without expecting the dependency to hold perfectly, but with the goal of measuring how many groups violate it and how many rows are affected. This gives a quantitative view of how clean and consistent the address information is in the Milan dataset.


In [32]:
fd4_violations = check_fd(
    MILANO,
    lhs_cols=["Tipo via", "Descrizione via", "Civico", "Codice via", "ZD"],
    rhs_col="Ubicazione",
)

fd4_violations.head()


FD ['Tipo via', 'Descrizione via', 'Civico', 'Codice via', 'ZD'] -> Ubicazione is violated in 628 out of 5470 groups.
Approximately 1455 rows (21.07%) are involved in violations.


Tipo via  Descrizione via       Civico  Codice via  ZD
VIA       CHIESE                60      1627        9     8
          SOLFERINO             12      1028        1     4
VLE       CONI ZUGNA            14      5110        7     4
VIA       DE CRISTOFORIS CARLO  2       1120        9     4
          TERTULLIANO           38      4094        4     4
Name: Ubicazione, dtype: int64

To apply automatic FD discovery algorithms such as TANE, CTANE and FD_Mine on the project data without exploding the search space, a reduced version of the Milan table is created.

Only five address-related attributes are kept:

`Tipo via`, `Descrizione via`, `Civico`, `Codice via`, `ZD`

These are exactly the columns used in the manual FD analysis. In addition, the columns are renamed to short identifiers A, B, C, D, E. This avoids technical issues in the DATADIQ implementations, which internally expect simple attribute names and may fail with long names containing spaces.


In [33]:
cols_fd = ["Tipo via", "Descrizione via", "Civico", "Codice via", "ZD"]

col_rename = {
    "Tipo via": "A",
    "Descrizione via": "B",
    "Civico": "C",
    "Codice via": "D",
    "ZD": "E",
}

MILANO_FD = (
    MILANO[cols_fd]
    .dropna()
    .astype(str)
    .rename(columns=col_rename)
)

MILANO_FD.head()


,A,B,C,D,E
0,ALZ,NAVIGLIO GRANDE,12,5144,6
1,ALZ,NAVIGLIO GRANDE,44,5144,6
2,ALZ,NAVIGLIO GRANDE,48,5144,6
3,ALZ,NAVIGLIO GRANDE,8,5144,6
4,ALZ,NAVIGLIO PAVESE,24,5161,6


The reduced and renamed table is saved as a separate CSV file.

This file contains only five columns (A–E) and is small enough for exact and approximate FD discovery algorithms to run efficiently. At the same time, it is still fully derived from the Milan dataset, so no external CSV files are introduced.


In [34]:
MILANO_FD.to_csv("MILANO_FD_TANE.csv", index=False)


The FD discovery algorithms TANE, CTANE and FD_Mine are imported from the DATADIQ package.

The system path is extended to include the local DATADIQ folder, so that `tane`, `ctane` and `fdtool` can be imported correctly.


In [35]:
import sys
sys.path.append("./DATADIQ")

from DATADIQ import tane, ctane
import fdtool


TANE is now executed on the reduced Milan table `MILANO_FD_TANE.csv`.

Since this table contains only five address attributes, the search space is small and the algorithm can terminate quickly. This makes it possible to apply TANE directly on the project data instead of on artificial examples.


In [36]:
source = "MILANO_FD_TANE.csv"
tane.compute(source)


List of all FDs:  [['D', 'A'], ['D', 'B'], ['AB', 'D'], ['BC', 'E'], ['CD', 'E']]
Total number of FDs found:  5


TANE discovers the following functional dependencies on the reduced Milan table (using the A–E column names):

D → A  
D → B  
A,B → D  
B,C → E  
C,D → E  

Using the renaming performed before, these can be mapped back to the original Milan attributes as follows.

A = `Tipo via`  
B = `Descrizione via`  
C = `Civico`  
D = `Codice via`  
E = `ZD`

This means:

`Codice via` → `Tipo via` and `Codice via` → `Descrizione via`  
Each street code corresponds to exactly one street type and one street name.

`Tipo via`, `Descrizione via` → `Codice via`  
The combination of street type and street name determines a unique street code.

`Descrizione via`, `Civico` → `ZD` and `Codice via`, `Civico` → `ZD`  
Both the pair (street name, civic number) and the pair (street code, civic number) determine a unique zone ZD.

These dependencies confirm and formalise the manual FD checks performed earlier, showing that the address information in the Milan dataset behaves like a clean and consistent addressing system.


FD_Mine, accessed through the `fdtool` module, is an evolution of TANE that uses additional pruning rules on the attribute lattice.

It is executed on the same reduced Milan CSV file. The goal is to verify whether a different FD discovery algorithm finds the same dependencies as TANE on the project data.


In [37]:
source = "MILANO_FD_TANE.csv"
fdtool.main(source)



Reading file: 
MILANO_FD_TANE.csv

Functional Dependencies: 
{D} -> {A}
{D} -> {B}
{B, C} -> {E}
{D, C} -> {E}
{B, A} -> {D}

Equivalences: 

Time (s): 0.0604
Row count: 6748
Attribute count: 5
Number of Equivalences: 0
Number of FDs: 5
Number of FDs checked: 54


/home/davide/Progetto_Data_Science/DATADIQ/fdtool_modules/GetFDs.py:24: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  else: return df.drop_duplicates(Candidate).count()[0];
/home/davide/Progetto_Data_Science/DATADIQ/fdtool_modules/GetFDs.py:24: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  else: return df.drop_duplicates(Candidate).count()[0];
/home/davide/Progetto_Data_Science/DATADIQ/fdtool_modules/GetFDs.py:24: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by posi

FD_Mine reports the same set of functional dependencies as TANE on the reduced Milan table.

In terms of the original attributes this means again:

`Codice via` determines both `Tipo via` and `Descrizione via`.  
The pair (`Tipo via`, `Descrizione via`) determines `Codice via`.  
The pairs (`Descrizione via`, `Civico`) and (`Codice via`, `Civico`) both determine `ZD`.

This agreement between TANE and FD_Mine strengthens the conclusion that the address-related functional dependencies discovered on the Milan dataset are robust and not an artefact of a single algorithm.


CTANE is an approximate version of TANE. It allows a limited number of violations for each candidate functional dependency, controlled by a threshold ε.

To keep the execution time reasonable, CTANE is applied to a random sample of the reduced Milan table, instead of the full dataset. This shows how approximate FD discovery can be used in principle on the same project data.


In [39]:
MILANO_FD_SMALL = MILANO_FD.sample(n=1000, random_state=0)
MILANO_FD_SMALL.to_csv("MILANO_FD_CTANE.csv", index=False)

source = "MILANO_FD_CTANE.csv"
ctane.compute(source, 0.1)


KeyboardInterrupt: 

Exception ignored in: 'zmq.backend.cython._zmq.Frame.__dealloc__'
Traceback (most recent call last):
  File "zmq/backend/cython/_zmq.py", line 179, in zmq.backend.cython._zmq._check_rc
KeyboardInterrupt: 


KeyboardInterrupt: 

In this notebook functional dependencies have been studied on the Milan public establishments dataset in two complementary ways.

First, the definition of FD was applied directly using pandas. This allowed to manually test and quantify natural dependencies such as:

{`Tipo via`, `Descrizione via`} → `Codice via`  
`Codice via` → `Tipo via` and `Codice via` → `Descrizione via`  
{`Tipo via`, `Descrizione via`, `Civico`, `Codice via`, `ZD`} → `Ubicazione`

Second, a reduced version of the Milan table with five address attributes was built and renamed to A–E. On this table, exact and approximate FD discovery algorithms from the DATADIQ package were applied:

TANE and FD_Mine both discovered the same set of meaningful FDs, which correspond exactly to the address relationships analysed manually. CTANE was executed on a sample of the same table to illustrate how approximate FD discovery can be used when a small number of violations is acceptable.

In this way the theoretical tools presented in the course (TANE, CTANE, FD_Mine) are demonstrated directly on the project data, and they are clearly connected to concrete data quality checks on the Milan dataset.
